In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/surveyres2023.csv' , index_col='ResponseId')
df_schema = pd.read_csv('data/surveyschema2023.csv')

In [3]:
pd.options.display.float_format = '{:.2f}'.format
pd.set_option('display.max_columns' , 84)
pd.set_option('display.max_rows' , 84)
pd.set_option('display.max_colwidth', None)

In [4]:
df_schema.loc[[15,16],'question']

15                             Including any education, how many years have you been coding in total?
16    NOT including education, how many years have you coded professionally (as a part of your work)?
Name: question, dtype: object

In [5]:
df_schema = pd.read_csv('data/surveyschema2023.csv' , index_col = 'qname' )

In [6]:
df_schema.drop(["S0","S1","S2","S3","S4","S5","S6","S7","Q120","Q310"] , inplace = True)

In [7]:
df_schema = df_schema.drop(columns=['type', 'selector' , 'qid' , 'force_resp'])

In [8]:
df_schema.head(5)

,question
qname,
MetaInfo,Browser Meta Info
MainBranch,"Which of the following options best describes you today? For the purpose of this survey, a developer is ""someone who writes code"". *"
Age,What is your age? *
Employment,Which of the following best describes your current employment status? Select all that apply.
RemoteWork,Which best describes your current work situation?


In [9]:
#df['Country'] = df['Country'].apply(lambda x:'USA' if x=='United States of America' else 'Iran' if x=='Iran, Islamic Republic of...' else x )
#esier way:
df['Country'] = df['Country'].replace({'United States of America':'United States', 'Iran, Islamic Republic of...': 'Iran'})
df['Country']

ResponseId
1                  NaN
2        United States
3        United States
4        United States
5          Philippines
             ...      
89180           Brazil
89181          Romania
89182           Israel
89183      Switzerland
89184             Iran
Name: Country, Length: 89184, dtype: object

In [10]:
filt = ((df['CompTotal']>96000) & (df['Country']=='United States') & (df['LanguageHaveWorkedWith'].str.contains('Python' , na=False)))
df.loc[filt,['CompTotal','LanguageHaveWorkedWith','Country']]

,CompTotal,LanguageHaveWorkedWith,Country
ResponseId,,,
2,285000.00,HTML/CSS;JavaScript;Python,United States
14,150000.00,C;C++;Python;Rust,United States
111,145000.00,HTML/CSS;JavaScript;Python;TypeScript,United States
122,200990.00,Go;HTML/CSS;JavaScript;PowerShell;Python;Rust,United States
138,150000.00,Python;R;SQL,United States
...,...,...,...
89079,150000.00,C;C++;Go;HTML/CSS;Java;JavaScript;MATLAB;Python;Rust;SQL;TypeScript,United States
89087,182333.00,Bash/Shell (all shells);HTML/CSS;JavaScript;Python;SQL;TypeScript,United States
89113,187000.00,C#;HTML/CSS;JavaScript;Python;Ruby;TypeScript,United States


In [11]:
#How to reorder columns:
#df['column1'],df['column2'] = df['column2'],df['column1']

In [12]:
'''There are couple of columns that aren't necessary that named knowledge_1,2 and ... and their existence just make our
dataframe nasty so let's create a function that will erase them for us.'''

def erase(*args):
    for arg in args:      
        for col in df.columns:
            if arg in col:
                df.drop(col, axis=1 , inplace = True)

In [13]:
#Before erasing
df.shape

(89184, 83)

In [14]:
erase('Knowledge_','Frequency_')
df.shape
#After erasing

(89184, 72)

In [15]:
del df['Q120']

In [16]:
df.loc[df['Country'].isin(['Iran','United States']),'Country'].value_counts()

Country
United States    18647
Iran               577
Name: count, dtype: int64

In [17]:
filt = (df['Country']=='Iran')

In [18]:
df.loc[filt, 'CompTotal']

ResponseId
75            5500.00
88                NaN
143     3600000000.00
177               NaN
228          20000.00
             ...     
88569             NaN
88636    110000000.00
88930             NaN
88939             NaN
89184   3300000000.00
Name: CompTotal, Length: 577, dtype: float64

In [19]:
filt = (df['ConvertedCompYearly'] < 2000000)
df = df.loc[filt]

In [20]:
df['RemoteWork'].unique()

array(['Remote', 'Hybrid (some remote, some in-person)', 'In-person', nan],
      dtype=object)

In [21]:
df['RemoteWork'].value_counts(normalize=True)

RemoteWork
Remote                                 0.44
Hybrid (some remote, some in-person)   0.42
In-person                              0.14
Name: proportion, dtype: float64

In [22]:
country_df = df.groupby(['Country'])

In [23]:
country_df['WebframeHaveWorkedWith'].apply(lambda x: x.str.contains('FastAPI', na=False).sum()).nlargest(25)

Country
United States                                           675
Germany                                                 265
United Kingdom of Great Britain and Northern Ireland    194
India                                                   159
Canada                                                  128
France                                                  126
Poland                                                  104
Brazil                                                   91
Netherlands                                              91
Spain                                                    88
Israel                                                   67
Australia                                                66
Italy                                                    63
Sweden                                                   56
Switzerland                                              45
Russian Federation                                       41
Mexico                          

In [24]:
filt = (df['WebframeHaveWorkedWith'].str.contains('FastAPI', na=False))
df_fastapi = df.loc[filt]

In [25]:
fastapi_grp = df_fastapi.groupby(['Country'])

In [26]:
fastapi_grp['ConvertedCompYearly'].median().nlargest(25)

Country
Montenegro                                             160634.00
United States                                          160000.00
Luxembourg                                             128507.00
Saudi Arabia                                           126306.00
Israel                                                 122948.00
Cyprus                                                 122317.00
Iceland                                                120674.00
Switzerland                                            114625.00
Swaziland                                              100310.00
Myanmar                                                100000.00
Australia                                               99156.00
China                                                   98482.00
Canada                                                  97028.50
Denmark                                                 94898.00
Ireland                                                 93168.00
United Kingdom of

# Modifying 2019 Data

In [27]:
df19 = pd.read_csv('data/data2019.csv', index_col= 'Respondent')

In [28]:
df19.dropna(axis='index', how= 'any', subset= ['DevType'], inplace=True)

In [29]:
df19['DevType'] = df19['DevType'].apply(lambda x:'Senior Executive (C-Suite, VP, etc.)' if 'Senior executive/VP' in x else x)

In [30]:
df19['DevType'] = df19['DevType'].apply(lambda x:x.split(';')[0])

In [31]:
df19['DevType'].value_counts().head(80)

DevType
Developer, back-end                              25635
Developer, full-stack                            11796
Database administrator                            5906
Academic researcher                               5684
Designer                                          5085
Data or business analyst                          4880
Developer, front-end                              4869
Developer, desktop or enterprise applications     3360
Data scientist or machine learning specialist     2998
Developer, mobile                                 2645
Senior Executive (C-Suite, VP, etc.)              2107
Student                                           1561
Developer, embedded applications or devices       1322
DevOps specialist                                  614
Developer, QA or test                              603
Engineering manager                                451
Developer, game or graphics                        391
Engineer, data                                     382
Sy

In [32]:
df19['Year'] = 2019

In [33]:
df19['SalaryUSD'].fillna(0, inplace=True)
df19['SalaryUSD'] = df19['SalaryUSD'].astype(int)

In [34]:
salary_bins = [0,60000,90000,150000,250000, float('inf')]
salary_labels = ['0-60000', '60001-90000', '90001-150000', '150001-250000', '250001+']

In [35]:
df19['salary_range'] = pd.cut(df19['SalaryUSD'], bins=salary_bins, labels=salary_labels, right=False)

In [36]:
df2019 = df19

In [37]:
df19= df19.loc[:, ['Country', 'SalaryUSD', 'DevType', 'salary_range', 'Year']]

## df 2023

In [38]:
df['Year'] = 2023

In [39]:
df['salary_range'] = pd.cut(df['ConvertedCompYearly'], bins=salary_bins, labels=salary_labels, right=False)

In [40]:
sal_country_grp = df.groupby(['Country', 'salary_range'], observed=False)

In [41]:
sal_country_grp.get_group(('United States', '90001-150000'))

,MainBranch,Age,Employment,RemoteWork,CodingActivities,EdLevel,LearnCode,LearnCodeOnline,LearnCodeCoursesCert,YearsCode,YearsCodePro,DevType,OrgSize,PurchaseInfluence,TechList,BuyNewTool,Country,Currency,CompTotal,LanguageHaveWorkedWith,LanguageWantToWorkWith,DatabaseHaveWorkedWith,DatabaseWantToWorkWith,PlatformHaveWorkedWith,PlatformWantToWorkWith,WebframeHaveWorkedWith,WebframeWantToWorkWith,MiscTechHaveWorkedWith,MiscTechWantToWorkWith,ToolsTechHaveWorkedWith,ToolsTechWantToWorkWith,NEWCollabToolsHaveWorkedWith,NEWCollabToolsWantToWorkWith,OpSysPersonal use,OpSysProfessional use,OfficeStackAsyncHaveWorkedWith,OfficeStackAsyncWantToWorkWith,OfficeStackSyncHaveWorkedWith,OfficeStackSyncWantToWorkWith,AISearchHaveWorkedWith,AISearchWantToWorkWith,AIDevHaveWorkedWith,AIDevWantToWorkWith,NEWSOSites,SOVisitFreq,SOAccount,SOPartFreq,SOComm,SOAI,AISelect,AISent,AIAcc,AIBen,AIToolInterested in Using,AIToolCurrently Using,AIToolNot interested in Using,AINextVery different,AINextNeither different nor similar,AINextSomewhat similar,AINextVery similar,AINextSomewhat different,TBranch,ICorPM,WorkExp,TimeSearching,TimeAnswering,ProfessionalTech,Industry,SurveyLength,SurveyEase,ConvertedCompYearly,Year,salary_range
ResponseId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
7,I am a developer by profession,35-44 years old,"Employed, full-time",Remote,Hobby;Contribute to open-source projects;Professional development or self-paced learning from online courses,Some college/university study without earning a degree,Friend or family member;Online Courses or Certification;Coding Bootcamp,NaN,Udemy,4,3,"Developer, full-stack","1,000 to 4,999 employees",I have little or no influence,NaN,Ask developers I know/work with;Read ratings or reviews on third party sites like G2 Crowd,United States,USD\tUnited States dollar,135000.00,Ada;Clojure;Elixir;Go;HTML/CSS;Java;JavaScript;Lisp;OCaml;Raku;Ruby;Scala;Swift;TypeScript;Zig,NaN,MariaDB;Microsoft SQL Server;MySQL;PostgreSQL;SQLite,Datomic,Amazon Web Services (AWS);Digital Ocean,Vercel,AngularJS;jQuery;Node.js;Phoenix;Ruby on Rails;Solid.js;Svelte;Vue.js,NaN,RabbitMQ;Spring Framework,NaN,Bun;Cargo;Chocolatey;Docker;Homebrew;Kubernetes;Make;Maven (build tool);npm;pnpm;Vite;Yarn,NaN,Emacs;IntelliJ IDEA;RubyMine;Visual Studio Code;Xcode,NaN,MacOS;Windows,MacOS,Jira,NaN,Slack,NaN,ChatGPT,NaN,GitHub Copilot,NaN,Stack Overflow,A few times per month or weekly,Yes,Less than once per month or monthly,Neutral,Neutral,Yes,Unfavorable,Improve accuracy in coding,Somewhat distrust,NaN,Writing code;Debugging and getting help,Learning about a codebase;Project planning;Writing code;Documenting code;Testing code;Committing and reviewing code;Deployment and monitoring;Collaborating with teammates,NaN,NaN,NaN,NaN,NaN,Yes,Individual contributor,4.00,Less than 15 minutes a day,15-30 minutes a day,Microservices;Automated testing;Continuous integration (CI) and (more often) continuous delivery,"Information Services, IT, Software Development, or other Technology",Appropriate in length,Easy,135000.00,2023,90001-150000
37,I am a developer by profession,55-64 years old,"Employed, full-time","Hybrid (some remote, some in-person)",Bootstrapping a business;Professional development or self-paced learning from online courses;Freelance/contract work,Some college/university study without earning a degree,"Books / Physical media;Online Courses or Certification;On the job training;Other online resources (e.g., videos, blogs, forum)",Formal documentation provided by the owner of the tech;How-to videos;Video-based Online Courses;Stack Overflow,Udemy;Pluralsight,33,15,"Developer, full-stack",100 to 499 employees,I have some influence,Investigate,Start a free trial;Visit developer communities like Stack Overflow,United States,USD\tUnited States dollar,130000.00,C#;HTML/CSS;JavaScript;SQL;TypeScript,C#;HTML/CSS;JavaScript;Python;Ruby;SQL;TypeScript,Elasticsearch;Microsoft SQL Server;SQLite,Cassandra;Elasticsearch;Microsof

In [42]:
# filt = (df['DevType']== 'Other (please specify):')
# df.drop(index=df[filt].index)
# df['DevType'].replace({'Research & Development role':'Academic researcher', 'Cloud infrastructure engineer':'DevOps specialist',
#                        'Security professional':'DevOps specialist', 'Hardware Engineer':'Engineering manager',
#                        'Project manager':'Product manager', 'Developer Experience': 'Developer, QA or test',
                       

In [43]:
devseries = sal_country_grp['DevType'].value_counts().loc['United States',['90001-150000', '150001-250000', '250001+']].head(80)
devseries

Country        salary_range   DevType                                      
United States  90001-150000   Developer, full-stack                            1771
                              Developer, back-end                               585
                              Developer, front-end                              279
                              Developer, desktop or enterprise applications     259
                              Developer, embedded applications or devices       172
                              Other (please specify):                           126
                              Data scientist or machine learning specialist      93
                              Developer, mobile                                  89
                              Engineer, data                                     81
                              DevOps specialist                                  74
                              Engineering manager                                66


In [44]:
sal_country_grp['WebframeHaveWorkedWith'].apply(lambda x: x.str.contains('FastAPI', na=False).sum() / len(x) * 100).loc[['United States', 'Canada']]

Country        salary_range 
United States  0-60000         2.68
               60001-90000     4.04
               90001-150000    4.92
               150001-250000   6.66
               250001+         6.95
Canada         0-60000         6.82
               60001-90000     4.96
               90001-150000    6.02
               150001-250000   7.93
               250001+         8.70
Name: WebframeHaveWorkedWith, dtype: float64

In [45]:
sal_country_grp[['LearnCodeCoursesCert', 'EdLevel']].value_counts(normalize=True).loc['United States', ['90001-150000', '150001-250000']].nlargest(60)

Country        salary_range   LearnCodeCoursesCert                   EdLevel                                                                           
United States  90001-150000   Udemy                                  Bachelor’s degree (B.A., B.S., B.Eng., etc.)                                         0.11
               150001-250000  Udemy                                  Bachelor’s degree (B.A., B.S., B.Eng., etc.)                                         0.08
                              Pluralsight                            Bachelor’s degree (B.A., B.S., B.Eng., etc.)                                         0.07
               90001-150000   Pluralsight                            Bachelor’s degree (B.A., B.S., B.Eng., etc.)                                         0.06
               150001-250000  Udemy;Pluralsight                      Bachelor’s degree (B.A., B.S., B.Eng., etc.)                                         0.06
               90001-150000   Udemy;Pluralsight      

In [46]:
df.rename(columns={'ConvertedCompYearly':'SalaryUSD'}, inplace=True)
df.index.names = ['Respondent']

In [47]:
df_devtype = df.loc[:, ['Country', 'SalaryUSD', 'DevType', 'salary_range', 'Year']]

In [48]:
df_devtype = pd.concat([df19, df_devtype], sort=False)

In [49]:
df_devtype['SalaryUSD'] = df_devtype['SalaryUSD'].astype(int)

In [50]:
df_devtype

,Country,SalaryUSD,DevType,salary_range,Year
Respondent,,,,,
2,Bosnia and Herzegovina,0,"Developer, desktop or enterprise applications",0-60000,2019
3,Thailand,8820,Designer,0-60000,2019
4,United States,61000,"Developer, full-stack",60001-90000,2019
5,Ukraine,0,Academic researcher,0-60000,2019
6,Canada,366420,Data or business analyst,250001+,2019
...,...,...,...,...,...
89163,Brazil,50719,"Developer, back-end",0-60000,2023
89167,Armenia,16917,"Developer, full-stack",0-60000,2023
89168,India,15752,"Developer, mobile",0-60000,2023


In [51]:
devgroup = df_devtype.groupby(['Year', 'Country', 'salary_range'], observed=False)
salgp = df_devtype.groupby(['Year', 'Country', 'DevType'])

In [52]:
devgroup['DevType'].value_counts().loc[[2019, 2023],'United States', '150001-250000'].head(60)

Year  Country        salary_range   DevType                                      
2019  United States  150001-250000  Developer, back-end                               915
                                    Developer, full-stack                             391
                                    Senior Executive (C-Suite, VP, etc.)              146
                                    Database administrator                            144
                                    Data scientist or machine learning specialist     138
                                    Developer, front-end                              134
                                    Data or business analyst                          110
                                    Developer, desktop or enterprise applications      88
                                    Designer                                           82
                                    Academic researcher                                70
                  

In [53]:
salgp['SalaryUSD'].median().loc[[2019,2023], 'United States', ['Data scientist or machine learning specialist',
                                                              'DevOps specialist', 'Developer, back-end', 'Developer, front-end',
                                                              'Developer, full-stack']]

Year  Country        DevType                                      
2019  United States  Data scientist or machine learning specialist   110000.00
                     DevOps specialist                               111000.00
                     Developer, back-end                             105000.00
                     Developer, front-end                             89000.00
                     Developer, full-stack                            92450.00
2023  United States  Data scientist or machine learning specialist   160000.00
                     DevOps specialist                               160000.00
                     Developer, back-end                             165000.00
                     Developer, front-end                            139510.00
                     Developer, full-stack                           140000.00
Name: SalaryUSD, dtype: float64

In [54]:
filt = (df['WebframeHaveWorkedWith'].str.contains('Django', na=False))
df.loc[filt, 'WebframeHaveWorkedWith'].count()

4141

In [55]:
filt = (df['WebframeHaveWorkedWith'].str.contains('FastAPI', na=False))
df.loc[filt, 'WebframeHaveWorkedWith'].count()

3088

In [56]:
df2019.rename(columns= {'WebFrameWorkedWith': 'WebframeHaveWorkedWith'}, inplace= True) 

In [57]:
filt = (df2019['WebframeHaveWorkedWith'].str.contains('Django', na=False))
df2019.loc[filt, 'WebframeHaveWorkedWith'].count()

7642

In [58]:
filt = (df2019['WebframeHaveWorkedWith'].str.contains('FastAPI', na=False))
df2019.loc[filt, 'WebframeHaveWorkedWith']

Series([], Name: WebframeHaveWorkedWith, dtype: object)

In [59]:
df_web19 = df2019.assign(WebframeHaveWorkedWith=df2019['WebframeHaveWorkedWith'].str.split(';')).explode('WebframeHaveWorkedWith')
df_web19 = df_web19.loc[:, ['Year', 'Country', 'WebframeHaveWorkedWith']]

In [60]:
df_web23 = df.assign(WebframeHaveWorkedWith=df['WebframeHaveWorkedWith'].str.split(';')).explode('WebframeHaveWorkedWith')
df_web23 = df_web23.loc[:, ['Year', 'Country', 'WebframeHaveWorkedWith']]

# WebFrame Work

In [61]:
df_framework = pd.concat([df_web19, df_web23], sort = False, ignore_index = True)

In [62]:
df_framework['Year'] = df_framework['Year'].astype(str)

In [63]:
filt = (df_framework['Year']=='2019')
df_framework[filt].count()

Year                      167235
Country                   167235
WebframeHaveWorkedWith    147334
dtype: int64

In [64]:
filt = (df_framework['WebframeHaveWorkedWith'].str.contains('FastAPI', na= False))
df_framework.loc[filt]

,Year,Country,WebframeHaveWorkedWith
167315,2023,Germany,FastAPI
167342,2023,Germany,FastAPI
167372,2023,Cyprus,FastAPI
167402,2023,Greece,FastAPI
167425,2023,France,FastAPI
...,...,...,...
307249,2023,Russian Federation,FastAPI
307287,2023,Russian Federation,FastAPI
307289,2023,India,FastAPI
307295,2023,Australia,FastAPI


In [65]:
df_framework.dropna(axis= 'index', how='any', subset=['WebframeHaveWorkedWith'], inplace= True)

In [66]:
filt = (df_framework['WebframeHaveWorkedWith'].str.contains('FastAPI', na= False)) & (df['Year']=='2019')
df_framework.loc[filt]

,Year,Country,WebframeHaveWorkedWith


In [67]:
frameworks_of_interest = ['Django', 'Flask', 'FastAPI']
df_framework_filtered = df_framework[df_framework['WebframeHaveWorkedWith'].isin(frameworks_of_interest)]

In [88]:
df_framework_filtered['Country'].count()

26337

In [68]:
filt = (df_framework['Year']=='2019')
df_framework_filtered.loc[filt]

,Year,Country,WebframeHaveWorkedWith
0,2019,Bosnia and Herzegovina,Django
3,2019,Ukraine,Django
5,2019,Ukraine,Flask
12,2019,India,Flask
20,2019,India,Django
...,...,...,...
167193,2019,Russian Federation,Django
167201,2019,United Kingdom,Django
167202,2019,United Kingdom,Flask
167211,2019,Argentina,Django


In [69]:
grp_framework = df_framework_filtered.groupby(['Year', 'Country', 'WebframeHaveWorkedWith']).size().reset_index(name='count')

In [70]:
django_df = grp_framework[grp_framework['WebframeHaveWorkedWith'] == 'Django']
flask_df = grp_framework[grp_framework['WebframeHaveWorkedWith'] == 'Flask']
fastapi_df = grp_framework[grp_framework['WebframeHaveWorkedWith'] == 'FastAPI']

In [78]:
django_flask = pd.merge(django_df, flask_df,  on=['Year', 'Country'], suffixes=('_django', '_flask'), how='outer')

In [80]:
comparison_df = pd.merge(django_flask, fastapi_df,  on=['Year', 'Country'], suffixes=('', '_fastapi'), how='outer')

In [81]:
comparison_df = comparison_df.rename(columns={'WebframeHaveWorkedWith': 'WebframeHaveWorkedWith_fastapi', 'count': 'count_fastapi'})

In [99]:
countries = ['United States', 'Canada', 'Germany', 'India', 'Australia']
filt = (comparison_df['Country'].isin(countries))
comparison_df.loc[filt]

,Year,Country,WebframeHaveWorkedWith_django,count_django,WebframeHaveWorkedWith_flask,count_flask,WebframeHaveWorkedWith_fastapi,count_fastapi
6,2019,Australia,Django,127.00,Flask,148.00,NaN,NaN
20,2019,Canada,Django,263.00,Flask,271.00,NaN,NaN
44,2019,Germany,Django,346.00,Flask,372.00,NaN,NaN
55,2019,India,Django,949.00,Flask,697.00,NaN,NaN
135,2019,United States,Django,1776.00,Flask,2037.00,NaN,NaN
148,2023,Australia,Django,89.00,Flask,89.00,FastAPI,66.00
162,2023,Canada,Django,173.00,Flask,180.00,FastAPI,128.00
181,2023,Germany,Django,244.00,Flask,296.00,FastAPI,265.00
189,2023,India,Django,270.00,Flask,247.00,FastAPI,159.00
267,2023,United States,Django,968.00,Flask,1160.00,FastAPI,675.00
